In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from scipy import stats
import datetime as dt
import random as rnd
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.plotting import ColumnDataSource,output_notebook
from bokeh.models import HoverTool
import math
from scipy.spatial.distance import euclidean

In [ ]:
avg_serv_time=[]
total_cours=[]
total_warehouses=[]
avg_busy_tim=[]
queue_orders=[]
avg_rating=[]
rating_means=[]
total_cust=[]
for number_whs in range(3,8):
    for number_cours in range(10,50,5):
        for number_cust in range(1000,4001,500): 
            n_of_customers=number_cust #int(input('The number of customers : '))
            n_of_couriers=number_cours #int(input('The number of couriers : '))
            n_of_warehouses=number_whs #int(input('The number of warehouses : '))
            average_order_waiting_limit=20*60 #int(input('Average Order waiting limit (in min) : '))*60
            
            order_prob=0.000005    #Order probabiities are distributed with Poisson (Lambda = 2 days)
    
            def calculateDistance(x1,y1,x2,y2):               #Calculates the hypotenuse distance of two points
                dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                return dist
            
            #Control every second whether an order was given or not.
            def Control_orders(objects,time):  #takes the exact time and calculate whether that customer give order at that time.
                customer_ids_gave_order=[]
                for i in range(n_of_customers):
                    if objects[i].order_time_hist != []:
                        if objects[i].order_time_hist[-1] == time :
                            customer_ids_gave_order.append(objects[i].cust_id)
                        
                return customer_ids_gave_order
            
            #Find the available courier in a specific time.
            def available_couriers(cour_objects):
                available_ones=[]
                for cour in range(n_of_couriers):
                    if(cour_objects[cour].availability == 1) & (cour_objects[cour].to_warehouse == 0):
                        available_ones.append(cour_objects[cour].cour_id)
                return available_ones
            
            
            #If a courier is available and there is not any order then the couriers are sent to the nearest warehouse to wait the next order.
            def nearest_warehouse(my_x,my_y):
                pt=None
                best_dist=9999999
                for num in range(n_of_warehouses):
                    hypotenuse_distance=calculateDistance(warehouse_objects[num].location_x,warehouse_objects[num].location_y,my_x,my_y)
                    if hypotenuse_distance< best_dist:
                        best_dist= hypotenuse_distance
                        pt=num
                return pt
            
            
            def order_served_time(order_placement,order_received):
                return (order_received-order_placement)
                
            
            #Finds the most suitable courier for a specific customer order in a specific time
            def finding_the_best_courier(custom,ava_cour):
                total_times_per_courier=[[0 for i in range(2)] for j in range(len(ava_cour))]
                for cours in range(len(ava_cour)):
                    total_times_per_warehouses=[]
                    for whs in range(n_of_warehouses):
                        taking_from_wh=calculateDistance(warehouse_objects[whs].location_x,warehouse_objects[whs].location_y,courier_objects[ava_cour[cours]].location_x,courier_objects[ava_cour[cours]].location_y)
                        delivery=calculateDistance(warehouse_objects[whs].location_x,warehouse_objects[whs].location_y,custom.location_x,custom.location_y)
                        total_times_per_warehouses.append(taking_from_wh+delivery)
                    total_times_per_courier[cours]=(min(total_times_per_warehouses)),total_times_per_warehouses.index(min(total_times_per_warehouses))
                total_times_per_courier=pd.DataFrame(total_times_per_courier)
                cour_ind=rnd.choice(np.where(total_times_per_courier[0] == np.array(total_times_per_courier[0]).min())[0])
                best_warehouse=total_times_per_courier[1][cour_ind]
                return cour_ind,best_warehouse
                
            
                
            def points_distance(point,other_points):
                return [ euclidean(point,other_points[j]) for j in range(len(other_points)) ]
            
            def maximum_distance(list_df):
                ds_max = max(list_df)
                idx = list_df.index(ds_max)
                return pts[idx]
                    
            
            
            class Warehouses:
                def __init__(self,wh_id):
                    self.wh_id=wh_id
                    self.location_x=0   #if one warehouse is selected, it is placed to the center of the area, but if there are more than 
                    self.location_y=0   #one warehouse than the warehouses are placed with distance maximizing condition.
                
            class Customers:
                def __init__(self,cust_id):
                    self.cust_id=cust_id
                    self.location_x=rnd.randint(-10000, 10000)
                    self.location_y=rnd.randint(-10000, 10000)
                    self.order_situation=0
                    self.order_waiting_time=np.random.normal(average_order_waiting_limit, 100, 1)[0]
                    self.order_time_hist=[]
                    self.order_arrival_time=[]
                    self.Rating=[]
                    
                    
                def give_order(self,time):
                    if (rnd.random()<=order_prob) & (self.order_situation == 0) : #One can not give the second order 
                        self.order_situation = 1                                  # before the last order is satisfied.
                        self.order_time_hist.append(time)
                        
                        
                def give_rating(self,order_time,arrival_time):
                    if(arrival_time-order_time<=self.order_waiting_time):
                        rating=5-(5*((arrival_time-order_time)/self.order_waiting_time))
                        if rating >4:
                            rating=5
                        else:
                            rating +=1
                        self.Rating.append(rating)
                    else:
                        self.Rating.append(1)
                
            class Couriers:
                def __init__(self,cour_id):
                    self.cour_id=cour_id
                    if(rnd.random()<0.2):
                        self.vehicle_type ='car'
                        self.capacity = 15
                    else:
                        self.vehicle_type = 'motorbike'
                        self.capacity = 3
                        
                    self.speed = np.random.normal(0.5, 0.05, 1)[0]
                    self.arrival_time=None                #the time between order accepting and warehouse arrival
                    self.order_distribution_time=None     #the time between package taken and customer distribution
                    self.order_taken_time=[]              #the time a courier accepts the order.
                    self.choosed_customer=None            #the customer for ordering
                    self.choosed_warehouse=None           #the warehouse that supplies the ordered product for a courier.
                    self.connected_warehouse=None         
                    self.location_x =0.0
                    self.location_y =0.0
                    self.in_warehouse=1
                    self.to_warehouse=0
                    self.order_serving_time=[]
                    self.courier_serving_time=[]
                    self.order_dist_time_history=[]
                    self.busy_for_warehouse=[]
                    self.availability=1                   #if courier is assigned an order availability is 0, else 1.
                
                def find_distance_time(self,your_x,your_y):
                    return int(calculateDistance(self.location_x,self.location_y,your_x,your_y) / (14*2*self.speed))
                
            
            #Creates warehouse, customer and courier objects.
            customer_objects=[]
            courier_objects=[]
            warehouse_objects=[]
            
            for ids in range(n_of_customers):
                customer_objects.append(Customers(ids))
            for ids in range(n_of_warehouses):
                warehouse_objects.append(Warehouses(ids))
            for ids in range(n_of_couriers):
                courier_objects.append(Couriers(ids))
                
                
            #With a heuristic optimization the warehouses are placed in the area by maximizing their distances to each other.
            if (n_of_warehouses > 1):
                N=50
                K=n_of_warehouses
                farthest_pts=[0]*K
                x=[ np.random.randint(-8000,8000) for p in range(N)]
                y=[ np.random.randint(-8000,8000) for p in range(N)]
                
                pts = [ [x[i],y[i]] for i in range(N)]
                P0 = pts[np.random.randint(0,N)]
                farthest_pts[0]=P0
                ds_tmp = points_distance(P0,pts)
                
                for i in range(1,K):
                    farthest_pts[i] = maximum_distance(ds_tmp)
                    ds_tmp2 = points_distance(farthest_pts[i],pts)
                    ds_tmp = [ min(ds_tmp[j],ds_tmp2[j]) for j in range(len(ds_tmp))]
                    
                for i in range(n_of_warehouses):
                    warehouse_objects[i].location_x=farthest_pts[i][0]
                    warehouse_objects[i].location_y=farthest_pts[i][1]
            
            #Distributes all couriers to warehouses equally so that couriers can start the simulation in a warehouse.
            for l in range(n_of_couriers): 
                k=l%n_of_warehouses
                courier_objects[l].location_x = warehouse_objects[k].location_x
                courier_objects[l].location_y = warehouse_objects[k].location_y  
                courier_objects[l].connected_warehouse=k
            
                
                
            def Simulation(runtime_in_second,order_prob,warehouse_objects,customer_objects,courier_objects):
                global order_queue
                order_queue=pd.DataFrame({"Customers_in_queue":[]})
                for second in range(runtime_in_second):
                    for j in range(n_of_customers):  # In every second customers give orders or not.
                        customer_objects[j].give_order(second)
            
                
                    x=Control_orders(customer_objects,second)
                    
                    if(x != []):
                        order_queue=order_queue.append(pd.DataFrame({"Customers_in_queue":x}),ignore_index = True)
                    
                    for j in range(n_of_couriers):
                        if(courier_objects[j].arrival_time == second):
                            if (courier_objects[j].availability == 0) & (courier_objects[j].to_warehouse == 0) :
                                cust=courier_objects[j].choosed_customer
                                courier_objects[j].arrival_time = None
                                courier_objects[j].order_distribution_time = second + 60 + courier_objects[j].find_distance_time(customer_objects[cust].location_x,customer_objects[cust].location_y)
                                courier_objects[j].location_x=warehouse_objects[courier_objects[j].choosed_warehouse].location_x
                                courier_objects[j].location_y=warehouse_objects[courier_objects[j].choosed_warehouse].location_y
                            else:
                                courier_objects[j].arrival_time = None
                                courier_objects[j].location_x=warehouse_objects[courier_objects[j].choosed_warehouse].location_x
                                courier_objects[j].location_y=warehouse_objects[courier_objects[j].choosed_warehouse].location_y
                                courier_objects[j].to_warehouse = 0
                                courier_objects[j].in_warehouse = 1
                        
                        if(courier_objects[j].order_distribution_time == second):
                            cust=courier_objects[j].choosed_customer
                            courier_objects[j].location_x=customer_objects[cust].location_x
                            courier_objects[j].location_y=customer_objects[cust].location_y
                            customer_objects[cust].order_arrival_time.append(courier_objects[j].order_distribution_time)
                            courier_objects[j].order_serving_time.append(order_served_time(customer_objects[cust].order_time_hist[-1],courier_objects[j].order_distribution_time))
                            courier_objects[j].courier_serving_time.append((courier_objects[j].order_distribution_time)-(courier_objects[j].order_taken_time[-1]))
                            courier_objects[j].order_dist_time_history.append(second)
                            #courier_objects[assigned_courier].order_taken_time=None
                            courier_objects[j].order_distribution_time = None
                            customer_objects[cust].give_rating(customer_objects[cust].order_time_hist[-1],customer_objects[cust].order_arrival_time[-1])
                            customer_objects[cust].order_situation=0
                            courier_objects[j].availability = 1
                            
                    
                    couriers_available = available_couriers(courier_objects)
                    
                    
                    if(list(order_queue['Customers_in_queue']) != []):
                        if(couriers_available != []):
                            aim_courier,aim_warehouse=finding_the_best_courier(customer_objects[int(order_queue.values[0][0])],couriers_available)
                            assigned_courier=couriers_available[aim_courier]
                            courier_objects[assigned_courier].choosed_warehouse=aim_warehouse
                            courier_objects[assigned_courier].choosed_customer=int(order_queue.values[0][0])
                            courier_objects[assigned_courier].order_taken_time.append(second+1)
                            courier_objects[assigned_courier].availability=0
                            courier_objects[assigned_courier].in_warehouse=0
                            courier_objects[assigned_courier].arrival_time=second + 5 + courier_objects[assigned_courier].find_distance_time(warehouse_objects[aim_warehouse].location_x,warehouse_objects[aim_warehouse].location_y)
                            order_queue=order_queue[1:]
                    
                    if second % 1000 == 30:
                        if (list(order_queue['Customers_in_queue']) == []) & (len(couriers_available) > (int(n_of_couriers/10))):
                            klm=[np.random.randint(0,n_of_couriers) for p in range(10)]
                            for j in range(n_of_couriers):
                                if ((courier_objects[j].in_warehouse == 0) & (courier_objects[j].availability == 1)) and (courier_objects[j].to_warehouse != 1):
                                    if ((courier_objects[j].location_x)**2 + (courier_objects[j].location_y)**2) > (9500**2) or (courier_objects[j].cour_id in klm):
                                        nearest_whs = nearest_warehouse(courier_objects[j].location_x,courier_objects[j].location_y)
                                        courier_objects[j].to_warehouse=1
                                        courier_objects[j].choosed_warehouse=nearest_whs
                                        courier_objects[j].arrival_time=second + 3 + courier_objects[j].find_distance_time(warehouse_objects[nearest_whs].location_x,warehouse_objects[nearest_whs].location_y)
                                        courier_objects[j].busy_for_warehouse.append(courier_objects[j].arrival_time-second)
            simulation_day=0.7
            for run in range(1):
                Simulation(int(simulation_day*24*60*60),order_prob,warehouse_objects,customer_objects,courier_objects)
            
            print(number_cours)
            print(number_whs)
            order_number_per_courier=[]
            customer_ratings=pd.DataFrame({"Ratings":[],"Seconds":[]})
            serving_times_per_courier=pd.DataFrame({"Order Times":[],"Order Dist. Time":[]})
            for ratio in range(n_of_couriers):
                order_number_per_courier.append(len(courier_objects[ratio].order_serving_time))
                if len(courier_objects[ratio].order_serving_time) != 0:
                    serving_times_per_courier=serving_times_per_courier.append(pd.DataFrame({"Order Times":courier_objects[ratio].order_serving_time,"Order Dist. Time":courier_objects[ratio].order_dist_time_history}),ignore_index = True)
            courier_idle_ratios=[]
            for i in range(n_of_couriers):
                courier_idle_ratios.append((sum(courier_objects[i].courier_serving_time)+sum(courier_objects[i].busy_for_warehouse))/int(simulation_day*24*60*60))
            for i in range(n_of_customers):
                if len(customer_objects[i].Rating) != 0:
                    avg_rating.append(np.mean(customer_objects[i].Rating))
            avg_serv_time.append(serving_times_per_courier['Order Times'].mean())
            total_cours.append(number_cours)
            total_warehouses.append(number_whs)
            total_cust.append(number_cust)
            queue_orders.append(len(order_queue))
            rating_means.append(np.mean(avg_rating))
            avg_busy_tim.append(np.array(courier_idle_ratios).mean())

In [ ]:
data=pd.DataFrame()
data['Average serving time']=avg_serv_time
data['Total Couriers']=total_cours
data['Total Warehouses']=total_warehouses
data['Total Customers']=total_cust
data['Average courier busy time']=avg_busy_tim
data['Average Rating']=rating_means
data['Orders in Queues']=queue_orders
data['Total Customers']=total_cust

In [7]:
data=pd.read_csv('simulation_result.csv')

In [8]:
pd.set_option('display.max_rows', data.shape[0]+1)
data

,Average serving time,Total Couriers,Total Warehouses,Total Customers,Average courier busy time,Average Rating,Orders in Queues
0,734.383871,10,3,1000,0.495327,2.486135,0
1,1540.173034,10,3,1500,0.868536,2.476192,2
2,9712.790068,10,3,2000,0.968925,2.459495,139
3,11479.167790,10,3,2500,0.981782,2.442216,250
4,14107.275930,10,3,3000,0.981332,2.423970,372
5,16314.684780,10,3,3500,0.983928,2.406653,502
6,19334.376940,10,3,4000,0.978492,2.390188,677
7,502.630363,15,3,1000,0.272850,2.401127,0
8,530.697115,15,3,1500,0.376182,2.415156,0
9,954.623824,15,3,2000,0.722300,2.416194,0
